# Deepcamp: Codelab 5

**In this tutorial we will cover**:

- Deployment of apps based on ML or DL models


**Author**:
- Alessio Devoto (alessio.devoto@uniroma1.it)


**Duration**: 50 mins 



1. Basic Gradio intro with a simple example (no ml function) + exercise

2. Set up an app that does speech to text


3. Set up an app that downloads a pretrained resnet and performs image recognition 

# Deployment of Machine Learning models 

Let's say you designed and trained a very cool and effective deep learning model. 

The model is now ready to be tested on a different data and users. What will you do ? 

- You could get in touch with someone who takes care of designing the infrastructure for your system ➡ expensive and time consuming
- You can use a framework for ML apps deployment and start it on any server (of course, you must own a server)

---

There are a few libraries we can use for this purpose: [Gradio](https://gradio.app/quickstart/), [StreamLit](https://streamlit.io/) and even come with UI features. If you want a lower level control (at the cose of more complexity), you can use [FastApi](https://fastapi.tiangolo.com/) or [Flask](https://flask.palletsprojects.com/en/2.3.x/).

Today, we are going to use Gradio, a new library developed by Huggingface for showcasing their own models.

Install necessary libraries as usual

In [16]:
%%capture
!pip install -U openai-whisper
!pip install gradio

## 1. Gradio basics

Gradio lets you create intuitive User Interfaces in plain Python, by simply listing panels, buttons, text areas etc...

All [components](https://gradio.app/docs/#components) must be initialized inside a gradio.Block, which represents an area of the interface.

Let's start with a simple example taken from [here](https://gradio.app/docs/#components).



In [ ]:
import gradio as gr

with gr.Blocks() as demo:   # main gradio block, always necessary

    # from here we can populate the UI as we wish
    gr.Markdown("DeepCamp: the best place in the **world**")
    
    with gr.Tab(label="Flip Text"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Flip")
   
    with gr.Tab(label="Flip Image"):
        with gr.Row():
            image_input = gr.Image()
            image_output = gr.Image()
        image_button = gr.Button("Flip")

demo.launch()

Of course, nothing happens if we click the buttons, as there is no function implemented yet.

Let's add simple functions which manipulate the input data and return an output.

In [ ]:
import numpy as np
import gradio as gr
import numpy as np


# a function that flips a string
def flip_text(x):
    return x[::-1]

# a function that flips an image
def flip_image(x):
    return np.fliplr(x)



with gr.Blocks() as demo:   # main gradio block, always necessary

    # from here we can populate the UI as we wish
    gr.Markdown("DeepCamp: the best place in the **world**")
    
    with gr.Tab(label="Flip Text"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Flip")
   
    with gr.Tab(label="Flip Image"):
        with gr.Row():
            image_input = gr.Image()
            image_output = gr.Image()
        image_button = gr.Button("Flip")

    # assign corresponding function to each button
    text_button.click(flip_text, inputs=text_input, outputs=text_output)

    # notice that the image is passed to the function as a numpy array
    image_button.click(flip_image, inputs=image_input, outputs=image_output)

demo.launch() # you can control the ports and the number of concurrent threads here


### Exercise 🏋: write a basic gradio demo

Write a gradio demo that given an image blurs it based on the input value provided by the user.

For the blur, use the provided function 



```
gaussian_filter(image, sigma)
```

where `image` is the image and `sigma` is the amount of blur.

Allow the user to select the amount of blur in the range (0,5) via a gradio Slider (see [here](https://gradio.app/docs/#slider-header))





In [ ]:
from scipy.ndimage import gaussian_filter

# your code here

In [ ]:
#@title Peek solution 👀

from scipy.ndimage import gaussian_filter


def blur(image, amount):
  return gaussian_filter(image, sigma=amount)

with gr.Blocks() as demo:   # main gradio block, always necessary

  # from here we can populate the UI as we wish
  gr.Markdown("DeepCamp: the best place in the **world**")

  blur_amount = gr.Slider(minimum=0, maximum=5)
  with gr.Row():
      image_input = gr.Image()
      image_output = gr.Image()
  blur_btn = gr.Button('Blur')

  # note how we pass the params
  blur_btn.click(blur, inputs=[image_input, blur_amount], outputs=image_output)


demo.launch()
    

## 2. Advanced Gradio Demo

This was an easy example, can we take another step and make things more interesting? 


**Current scenario:** we developed a speech to text model which is able to trascribe human speech, and we want to make this service available to all our colleagues.


### 2.1 Whisper
Meet OpenAI's Whisper!

![whisper](https://openaicom.imgix.net/d9c13138-366f-49d3-b8bd-cb3f5a973a5b/asr-summary-of-model-architecture-desktop.svg?fm=auto&auto=compress,format&fit=min&w=1919&h=1551)


[Whisper](https://arxiv.org/pdf/2212.04356.pdf) is an automatic speech recognition (ASR) system trained on *680.000 hours of multilingual and multitask* supervised data collected from the web 🤯. 

The Whisper architecture is a simple end-to-end approach, implemented as an encoder-decoder Transformer. As you may know, Encoder and Decoder blocks are neural networks based on the attention mechanism. 

As you can see on the [official repository](https://github.com/openai/whisper) they trained the model in different "sizes": 

```
tiny, small, base, medium, large, large-v2

In [4]:
from torchaudio.utils import download_asset
import IPython

speech_file = download_asset("tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav")
IPython.display.Audio(speech_file)

In [22]:
!cp /root/.cache/torch/hub/torchaudio/tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav /content/

In [14]:
import whisper

model = whisper.load_model("tiny") # we load the tiny version
result = model.transcribe(
    speech_file, 
    language='en', 
    temperature = 0,    # where have we met this before ? 🤔
    task='transcribe') # or translate
print(result["text"])

/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Sum caminho risulta Isabelle


In [9]:
result

{'text': ' I had that curiosity beside me at this moment.',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.2,
   'text': ' I had that curiosity beside me at this moment.',
   'tokens': [50364,
    286,
    632,
    300,
    18769,
    15726,
    385,
    412,
    341,
    1623,
    13,
    50524],
   'temperature': 0.0,
   'avg_logprob': -0.2618454419649564,
   'compression_ratio': 0.8679245283018868,
   'no_speech_prob': 0.01015368103981018}],
 'language': 'en'}

Ok, so we know how Whisper works now. Can we integrate this into a Gradio demo ?


### 2.2 Whisper + Gradio

In [ ]:
import gradio as gr

def whisper_transcribe(audio, temp, lang):

  if lang == 'Detect':
    lang = None

  result = model.transcribe(
    audio, 
    language=lang, 
    temperature=temp,    
    task='transcribe')

  return result["text"]


with gr.Blocks() as demo:   # main gradio block, always necessary

  # from here we can populate the UI as we wish
  gr.Markdown("DeepCamp: the best AI camp in the **world**")
  with gr.Row():
      temp = gr.Slider(minimum=0, maximum=1)
      lang = gr.Radio(choices=['en', 'it', 'pt', 'es', 'fr', 'Detect'], value='Detect')
  with gr.Tab("Upload"):
      audio_up = gr.Audio(source="upload", type='filepath')
      transcr_up = gr.Button('Trascribe')

  with gr.Tab("Record"):
      audio_mic = gr.Audio(source="microphone", type='filepath')
      transcr_mic = gr.Button('Trascribe')
  
  trascript = gr.Text()
  
  
  transcr_up.click(
      whisper_transcribe, 
      inputs=[audio_up, temp, lang], 
      outputs=trascript)
  
  transcr_mic.click(
      whisper_transcribe, 
      inputs=[audio_mic, temp, lang], 
      outputs=trascript)


demo.launch(debug=True)

## 3. Final Exercise 🔥

Build a gradio demo that predicts the content of an image using a pretrained ResNet.

In [22]:
from torchvision.models import resnet34, ResNet34_Weights
from PIL import Image
import requests

labels = eval(requests.get('https://raw.githubusercontent.com/alessiodevoto/deepers/main/data/imagenet1000_clsidx_to_labels.txt').text)
resnet34 = resnet34(weights=ResNet34_Weights.DEFAULT)
resnet34.eval()
preprocess = ResNet34_Weights.DEFAULT.transforms()

In [25]:
import gradio as gr
import torch


def classify(img):
  image = Image.open(img)
  img_transformed = preprocess(image)
  logits = resnet34(img_transformed.unsqueeze(0))
  predictions = torch.nn.functional.softmax(logits, dim=1)[0] 
  confidences = {labels[i]: float(predictions[i]) for i in range(1000)}
  return confidences


with gr.Blocks() as demo:   # main gradio block, always necessary
  gr.Markdown("DeepCamp: the best AI camp in the **world**")
  
  with gr.Row():
      input_image = gr.Image(type='filepath')
      pred = gr.Label(num_top_classes=3)
  with gr.Row():
    pred_btn = gr.Button('Classify')

  pred_btn.click(
      classify, 
      inputs=input_image, 
      outputs=pred)
  
demo.launch(debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
